# 1\. Conversion function
Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
def converti(a):
    conv_func=[int, bin, hex]
    b=[f(a) for f in conv_func]
    return b


print(converti(0b1111))
print(converti(15))
print(converti(0xf))

[15, '0b1111', '0xf']
[15, '0b1111', '0xf']
[15, '0b1111', '0xf']


# 2\. 32 bit float conversion
Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [2]:

def converti(a):
    try:
        sgn= +1 if a[0]=="0" else -1
       
       #esponente
        exp=int(a[1:9],2) 
        exp-=127
        
       #mantissa
        man=a[-23:]
        t=[2**(-i-1)*int(man[i]) for i in range(len(man))]  #calcola potenze di 2
        man=1+sum(t)
        
       #numero finale
        num=man*(2**exp)*sgn
        print(num)
    
    except: print("inserire numero corretto")
 


a="11000000010010001111010111000011"
b="00000011111000000000000000000000"
converti(a)
converti(b)
converti("ciao")

-3.140000104904175
1.316553672920962e-36
inserire numero corretto


# 3\. Underflow & overflow
Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [7]:

print("Lower limit: ")  # con denormalized numbers
a=1.
while True:          
    if a/2>0 : a/=2  #cicla finché diverso da 0
    else:
        print(a)
        break


print("Upper limit: ")   
b=1.
while True:   #cicla finché minore di inf
    if b*2 != float("inf") : b*=2
    else:
        print(b)
        break

Lower limit: 
5e-324
Upper limit: 
8.98846567431158e+307


# 4\. Write a program to determine the machine precision 

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [17]:
a=1.
b=1.
while b.hex()!=(b+a).hex():
    if a/2>0 : a/=2
    else: break

print(a)

print()
import sys
print(sys.float_info)


1.1102230246251565e-16

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [3]:
import math

""" Sotto radice si sta facendo la differenza tra un numero molto grande (b^2) e uno molto piccolo (4ac), quindi il 
risultato è approssimabile a b. La soluzione del tipo -b+sqrt() è quindi una differenza tra due numeri molto simili,
e ciò comporta i problemi di approssimazione maggiore, mentre la soluzione -b-sqrt() comporta un errore minore. 
Moltiplicando e dividendo numeratore e denominatore per il fattore indicato si può ricavare la soluzione -b+sqrt() del 
primo metodo riportandosi al conto di -b-sqrt(), che riduce l'errore. Per cui combinando le due soluzioni si ottiene 
il risultato corretto nel caso generale"""

def sol1(a,b,c):   #Formula classica con numeratore/denominatore
    n1=-b + math.sqrt(b**2-4*a*c)
    n2=-b - math.sqrt(b**2-4*a*c)
    d=(2*a)
    return [n1/d,n2/d]

def sol2(a,b,c):  #Prodotto di -b-+sqrt(b^2-4ac) calcolato analiticamente
    n= 2*c
    d1=(-b + math.sqrt(b**2-4*a*c))
    d2=(-b - math.sqrt(b**2-4*a*c))
    return [n/d1,n/d2]

def sol3(a,b,c):    #Metodo corretto: scelta di un risultato da sol1 e uno da sol2
    return[sol1(a,b,c)[1],sol2(a,b,c)[1]]



a=0.001
b=1000
c=0.001

sol=[sol1,sol2,sol3]
    
for s in sol:
    s_1=s(a,b,c)[0]
    s_2=s(a,b,c)[1]
    print("s1="+str(s_1)+";\t s2="+str(s_2)+ ";\t s1*s2="+str(s_1*s_2)) #Come controllo s1*s2 = c/a = 1
        


s1=-9.999894245993346e-07;	 s2=-999999.999999;	 s1*s2=0.9999894245983346
s1=-1000010.5755125057;	 s2=-1.000000000001e-06;	 s1*s2=1.0000105755135058
s1=-999999.999999;	 s2=-1.000000000001e-06;	 s1*s2=1.0


$$ Sol1= \frac{-b\pm \sqrt{b^2-4ac}}{2a}$$
$$ Sol2= \frac{2c}{-b\mp \sqrt{b^2-4ac}}$$
$$ Sol3= \frac{-b - \sqrt{b^2-4ac}}{2a} \, and \, \frac{2c}{-b - \sqrt{b^2-4ac}}$$


# 6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:

def f(x):          #funzione f=x*(x-1)
    return x*(x-1)

def derivata(z,d):         #calcolo della derivata 
    return (f(z+d)-f(z))/d

# a)---------------------------------------------------------------------------------------
z=1.
d=10**(-2)

print("Derivata python= "+str(derivata(z,d)))
print("Derivata analitica= 1")
print("I risultati sono diversi sia a causa del round off error del valore float d,\n\
e principalmente dal suo valore che rappresenta un incremento relativamente grande")

# b)---------------------------------------------------------------------------------------
print()
for i in range(1,8):
    d=10**(-i*2)
    print("d= "+format(d) + "; derivata= %2.10f"%derivata(z,d)+ "; accuratezza= "+format(abs(1-derivata(z,d))))
    
print()
print("L'accuratezza diventa migliore diminuendo il valore dell'incremento delta fino al \n\
valore di 10^-8, ovvero sqrt(epsilon) dove epsilon è l'errore della macchina nell'approssimazione \n\
dei float, per poi aumentare anche diminuendo ulteriormente l'incremento.")



Derivata python= 1.010000000000001
Derivata analitica= 1
I risultati sono diversi sia a causa del round off error del valore float d,
e principalmente dal suo valore che rappresenta un incremento relativamente grande

d= 0.01; derivata= 1.0100000000; accuratezza= 0.010000000000000897
d= 0.0001; derivata= 1.0001000000; accuratezza= 9.999999988985486e-05
d= 1e-06; derivata= 1.0000009999; accuratezza= 9.99917733279787e-07
d= 1e-08; derivata= 1.0000000039; accuratezza= 3.922528746258536e-09
d= 1e-10; derivata= 1.0000000828; accuratezza= 8.284037100736441e-08
d= 1e-12; derivata= 1.0000889006; accuratezza= 8.890058334132256e-05
d= 1e-14; derivata= 0.9992007222; accuratezza= 0.0007992778373491216

L'accuratezza diventa migliore diminuendo il valore dell'incremento delta fino al 
valore di 10^-8, ovvero sqrt(epsilon) dove epsilon è l'errore della macchina nell'approssimazione 
dei float, per poi aumentare anche diminuendo ulteriormente l'incremento.


# 7\. Integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [31]:
#a)-----------------------
import math

def semic(x):
    return math.sqrt(1-x**2)

def integral(N,f,a,b):
    sum=0
    for i in range(N+1):
        sum+= (2/N)*f(a+2*i/N)
    
    return sum
   
   
print("N= 100")
print("Computed integral value: %f"%integral(100,semic,-1,1))
print("True integral value: %f"% (math.pi/2))
print("Relative error: %f"% abs(integral(100,semic,-1,1)/(math.pi/2)-1))

N= 100
Computed integral value: 1.569134
True integral value: 1.570796
Relative error: 0.001058


In [20]:
%timeit integral(100,semic,-1,1)
%timeit integral(1300000,semic,-1,1) #N può essere aumentata anche di 4 ordini di grandezza stando sotto ad un secondo di esecuzione

70.6 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
908 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
print("N= 1300000, 1 sec di esecuzione")
print("Computed integral value: "+format(integral(1300000,semic,-1,1)))
print("True integral value: " +format((math.pi/2)))
print("Relative error: " +format(abs(integral(1300000,semic,-1,1)/(math.pi/2)-1))) 

N= 1300000, 1 sec di esecuzione
Computed integral value: 1.5707963256729667
True integral value: 1.5707963267948966
Relative error: 7.142427760342684e-10
